!python -m spacy download pt_core_news_sm

In [1]:
import numpy as np
import pandas as pd
import spacy
import string
import matplotlib.pyplot as plt
import tensorflow as tf
from nltk import tokenize
from tensorflow import keras
from keras.layers import Input, LSTM, Dense, Embedding
from keras.models import Model
import tensorflow_datasets as tfds
import random
from sklearn.model_selection import train_test_split
from tensorflow.python.client import device_lib
#import pt_core_news_sm

In C:\Users\renat\Anaconda3\envs\TCC\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\renat\Anaconda3\envs\TCC\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\renat\Anaconda3\envs\TCC\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\renat\Anaconda3\envs\TCC\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\renat\Anaconda3\envs\TCC\lib\site-packages\m

In [2]:
#nlp = pt_core_news_sm.load()
nlp = spacy.load('pt_core_news_sm')

In [3]:
data = pd.read_json('tcc1.json')

In [4]:
stopwords = spacy.lang.pt.STOP_WORDS
pontos = string.punctuation

In [5]:
def frase_limpa(texto):
    palavra = []
    texto = texto.lower()
    texto = nlp(texto)
    for p in texto:
        palavra.append(p.text)
    texto = [p for p in palavra if p not in stopwords and p not in pontos]
    texto = ' '.join([i for i in texto])
    return texto

In [6]:
noticia_frase = []
for i in range(len(data.texto)):
    noticia_frase.append(frase_limpa(data.texto[i]))

In [7]:
token_frase = []
for t in noticia_frase:
    token_frase.append(t.split(" "))

In [8]:
len(noticia_frase[0])

5037

In [9]:
tokens = []
for t in token_frase:
    for i in range(len(t)):
        if len(t[i]) > 1:
            tokens.append(t[i])

In [10]:
len(tokens)

78244

In [11]:
tokens = np.unique(tokens)

In [12]:
len(tokens)

14819

In [13]:
titulo_frase = []
for i in range(len(data.título)):
    titulo_frase.append(frase_limpa(data.título[i]))

In [14]:
token_t = []
for t in titulo_frase:
    token_t.append(t.split(" "))

In [15]:
tokens_titulos = []
for t in token_t:
    for i in range(len(t)):
        if len(t[i]) > 0:
            tokens_titulos.append(t[i])

In [16]:
len(tokens_titulos)

2451

In [17]:
tokenizador = tfds.features.text.SubwordTextEncoder.build_from_corpus(noticia_frase, target_vocab_size=2**10)

In [18]:
frase_input = [tokenizador.encode(frase) for frase in noticia_frase]

In [19]:
num_encoder_tokens = len(tokens)
num_decoder_tokens = len(tokens_titulos)

In [20]:
for i in tokens:
    token_frase = tokenizador.encode(i)
for i in tokens_titulos:
    token_titulo = tokenizador.encode(i)

In [21]:
tokens

array(['"abordagem', '"bebês', '"como', ..., '\u200b\u200bem', '”dias',
       '”vídeo'], dtype='<U93')

In [22]:
input_token_index = dict([(i, tokenizador.encode(i)) for i in tokens])
target_token_index = dict([(i, tokenizador.encode(i)) for i in tokens_titulos])

In [23]:
tam_max = max([len(frase)] for frase in frase_input)
tam_max = tam_max[0]

In [24]:
frase_input = tf.keras.preprocessing.sequence.pad_sequences(frase_input, value = 0, padding='post', maxlen=tam_max)

In [25]:
titulo_input = [tokenizador.encode(titulo) for titulo in titulo_frase]

In [26]:
titulo_max = max([len(titulo)] for titulo in titulo_input)
titulo_max = titulo_max[0]

In [27]:
print(tam_max)
print(titulo_max)

7069
44


In [28]:
titulo_input = tf.keras.preprocessing.sequence.pad_sequences(titulo_input, value = 0, padding='post', maxlen=titulo_max)

In [29]:
vocab_size = tokenizador.vocab_size
latent_dim = 128
emb_dim = 25
nb_filter = 25
ffn_units = 128
dropout_rate = 0.2
nb_epochs = 100

In [30]:
encoder_input_data = np.zeros(
    (len(frase_input), tam_max, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(frase_input), titulo_max, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(titulo_input), titulo_max, num_decoder_tokens),
    dtype='float32')

MemoryError: Unable to allocate 112. GiB for an array with shape (287, 7069, 14819) and data type float32

In [ ]:
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

In [ ]:
encoder_input = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(emb_dim, return_state=True)
encoder_output, state_h, state_c = encoder(encoder_input)
encoder_state = [state_h, state_c]


#encoder_input = Input(shape=(None,))
#x = Embedding(vocab_size, latent_dim)(encoder_input)
#x, state_h, state_c = LSTM(latent_dim, return_state=True)(x)
#encoder_states = [state_h, state_c]

In [ ]:
print(encoder_input.shape)

In [ ]:
decoder_input = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(emb_dim, return_sequences=True, return_state=True)
decoder_output, _, _ = decoder_lstm(decoder_input, initial_state=encoder_state)
decoder_dense = Dense(titulo_max, activation='softmax')
decoder_output = decoder_dense(decoder_output)



#decoder_inputs = Input(shape=(None,))
#x = Embedding(vocab_size, latent_dim)(decoder_inputs)
#x = LSTM(latent_dim, return_sequences=True)(x, initial_state=encoder_states)
#decoder_outputs = Dense(vocab_size, activation='softmax')(x)

In [ ]:
print(decoder_lstm)
print(decoder_output.shape)

In [ ]:
print(decoder_input.shape)

In [ ]:
model = Model([encoder_input, decoder_input], decoder_output)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
model.fit([encoder_input_data, decoder_target_data], batch_size=64, epochs=nb_epochs, validation_split=0.2, verbose=2)